In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import albumentations as A
from albumentations.pytorch import transforms, ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms

from sklearn.metrics import f1_score, accuracy_score

In [2]:
# !pip3 install timm albumentations

In [3]:
# Configs
config = {
    "SEED": 777,
    "SIZE": 512,
    "CSV": "./train_kfold.csv",
    
    "FOLD": 0,
    "BATCH_SIZE": 8,
    "LEARNING_RATE": 0.001,
    "EPOCHS": 30,
    "N_WORKERS": 4,
    
    "MODEL": "tf_efficientnet_b7",
    "MODEL_SAVE": "./b7_model",
    "MODEL_SAVE_PREFIX": "b7_777_",
    
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

In [4]:
if not os.path.exists(config["MODEL_SAVE"]):
    os.makedirs(config["MODEL_SAVE"], exist_ok=True)

In [5]:
random.seed(config["SEED"])
torch.cuda.manual_seed(config["SEED"])
torch.manual_seed(config["SEED"])
torch.cuda.empty_cache()

In [6]:
class CustomDataset(Dataset):
    def __init__(self,
                 data_path,
                 size,
                 transform=None,
                 fold=0,
                 mode="train"):
        self.csv = pd.read_csv(data_path)
        if mode == "train":
            self.csv = self.csv[self.csv['kfold'] != fold]
        elif mode == "validation":
            self.csv = self.csv[self.csv['kfold'] == fold]
        
        self.path = self.csv['path'].to_list()
        self.labels = self.csv['encoded_label'].to_list()
        self.transform = transform
        self.size = size
        self.mode = mode
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, idx):        
        # Image
        image = cv2.imread(self.path[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(self.size)(image=image)['image']
        
        # Only test mode
        if self.mode == "test":
            return {
                'image': image
            }
        
        # Label
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)
        
        return {
            'image': image,
            'label': label
        }

In [7]:
def create_train_transforms(size=512):
    return A.Compose([
        A.Resize(size, size),
        A.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1),
            contrast_limit=(-0.1, 0.1),
            p=0.3
        ),
        ToTensorV2()
    ])

In [8]:
def create_validation_transforms(size=128):
    return A.Compose([
        A.Resize(size, size),
        ToTensorV2()
    ])

In [9]:
train_dataset = CustomDataset(
    data_path = config["CSV"],
    size = config["SIZE"],
    transform = create_train_transforms,
    fold = config["FOLD"],
    mode = "train"
)
validation_dataset = CustomDataset(
    data_path = config["CSV"],
    size = config["SIZE"],
    transform = create_validation_transforms,
    fold = config["FOLD"],
    mode = "validation"
)
train_loader = DataLoader(
    dataset = train_dataset,
    shuffle = True,
    batch_size = config["BATCH_SIZE"],
    num_workers = config["N_WORKERS"]
)
validation_loader = DataLoader(
    dataset = validation_dataset,
    shuffle = False,
    batch_size = config["BATCH_SIZE"],
    num_workers = config["N_WORKERS"]
)

In [10]:
# Create Model (Using timm library)
class CustomModel(nn.Module):
    def __init__(self, model_name = None, pretrained = True, num_classes = 88):
        super(CustomModel, self).__init__()
        self.model = timm.create_model(model_name,
                                       pretrained = pretrained,
                                       num_classes = num_classes)
    def forward(self, x):
        return self.model(x)

In [11]:
model = CustomModel(config["MODEL"]).to(config["DEVICE"])

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=config["LEARNING_RATE"])
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

In [13]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [14]:
best_loss, best_pred = float('inf'), 0
for epoch in range(config["EPOCHS"]):
    train_loss = 0
    total_train_loss = 0
    train_data_cnt = 0
    train_pred=[]
    train_y=[]
    model.train()
    pbar = tqdm(train_loader, total=len(train_loader))
    for batch in pbar:
        optimizer.zero_grad()
        x = torch.tensor(batch['image'], dtype=torch.float32, device=config["DEVICE"])
        y = torch.tensor(batch['label'], dtype=torch.long, device=config["DEVICE"])
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_train_loss += loss.item() / len(train_loader)
        train_loss = train_loss * train_data_cnt + loss.item()
        train_data_cnt += 1
        train_loss /= train_data_cnt
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        pbar.set_postfix({
            "epoch": f"{epoch}/{config['EPOCHS']}",
            "train_loss" : f"{train_loss:.5f}",
            "total_train_loss": f"{total_train_loss:.5f}"
        })
        
    pbar.close()
    train_f1 = score_function(train_y, train_pred)
    print(f'TRAIN\tf1 : {train_f1:.5f}')
    
    validation_loss = 0
    total_validation_loss = 0
    validation_data_cnt = 0
    validation_pred=[]
    validation_y=[]
    model.eval()
    pbar = tqdm(validation_loader, total=len(validation_loader))
    with torch.no_grad():
        for batch in pbar:
            x = torch.tensor(batch['image'], dtype=torch.float32, device=config["DEVICE"])
            y = torch.tensor(batch['label'], dtype=torch.long, device=config["DEVICE"])
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)
            total_validation_loss += loss.item() / len(validation_loader)
            validation_loss = validation_loss * validation_data_cnt + loss.item()
            validation_data_cnt += 1
            validation_loss /= validation_data_cnt
            validation_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            validation_y += y.detach().cpu().numpy().tolist()
            pbar.set_postfix({
                "epoch": f"{epoch}/{config['EPOCHS']}",
                "val_loss" : f"{validation_loss:.5f}",
                "total_val_loss": f"{total_validation_loss:.5f}"
            })
        pbar.close()
    val_f1 = score_function(validation_y, validation_pred)
    print(f'VAL\tf1 : {val_f1:.5f}')
    
    # Update about Loss
    if best_loss > validation_loss:
        best_loss = validation_loss
        torch.save({
            "epoch": epoch,
            "loss": validation_loss,
            "score": val_f1,
            "state_dict": model.state_dict()
        }, f"{config['MODEL_SAVE']}/{config['MODEL_SAVE_PREFIX']}_best.pth")
    
    torch.save({
        "epoch": epoch,
        "loss": validation_loss,
        "score": val_f1,
        "state_dict": model.state_dict()
    }, f"{config['MODEL_SAVE']}/{config['MODEL_SAVE_PREFIX']}_{epoch}.pth")
    
    # Update about Score
    #if best_score < val_f1:
    #    best_score = val_f1

100%|██████████| 401/401 [01:43<00:00,  3.89it/s, epoch=0/30, train_loss=1.34686, total_train_loss=1.34686]


TRAIN	f1 : 0.15419


100%|██████████| 134/134 [00:07<00:00, 17.25it/s, epoch=0/30, val_loss=0.91163, total_val_loss=0.91163]


VAL	f1 : 0.17926


100%|██████████| 401/401 [01:41<00:00,  3.93it/s, epoch=1/30, train_loss=0.80049, total_train_loss=0.80049]


...

VAL	f1 : 0.44227


100%|██████████| 401/401 [01:42<00:00,  3.92it/s, epoch=29/30, train_loss=0.11954, total_train_loss=0.11954]


TRAIN	f1 : 0.86647


100%|██████████| 134/134 [00:07<00:00, 17.14it/s, epoch=29/30, val_loss=0.66838, total_val_loss=0.66838]


VAL	f1 : 0.51935
